In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import numpy as np
from PIL import Image
import timm  # For Vision Transformer (ViT)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from glob import glob

# Define dataset class
class RAFDBDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob(os.path.join(root_dir, "*.jpg"))
        self.labels = [int(os.path.basename(p).split("_")[0]) for p in self.image_paths]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load Dataset
dataset = RAFDBDataset(root_dir="path_to_rafdb", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Feature Extractor (CNN + ViT)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])

        self.resnet = models.resnet50(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()  # Remove classification head

        self.fc = nn.Linear(1280 + 2048 + 768, 512)  # Concatenate features

    def forward(self, x):
        feat1 = self.efficientnet(x).flatten(start_dim=1)
        feat2 = self.resnet(x).flatten(start_dim=1)
        feat3 = self.vit(x)
        combined_feat = torch.cat([feat1, feat2, feat3], dim=1)
        return self.fc(combined_feat)

# Pose Normalization (STN)
class STN(nn.Module):
    def __init__(self):
        super(STN, self).__init__()
        self.localization = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=7),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_loc = nn.Sequential(
            nn.Linear(32 * 53 * 53, 128),
            nn.ReLU(),
            nn.Linear(128, 6)
        )
        self.fc_loc[2].weight.data.fill_(0)
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 32 * 53 * 53)
        theta = self.fc_loc(xs).view(-1, 2, 3)
        grid = torch.nn.functional.affine_grid(theta, x.size())
        return torch.nn.functional.grid_sample(x, grid)

# Define Model with STN and Feature Extraction
class PoseAwareFER(nn.Module):
    def __init__(self):
        super(PoseAwareFER, self).__init__()
        self.stn = STN()
        self.feature_extractor = FeatureExtractor()
        self.classifier = nn.Linear(512, 7)  # Assuming 7 classes in RAF-DB

    def forward(self, x):
        x = self.stn(x)
        features = self.feature_extractor(x)
        return self.classifier(features)

# Train Function
def train_model(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}, Accuracy: {accuracy:.2f}%")

    return model

# Train the model
model = PoseAwareFER()
trained_model = train_model(model, train_loader, test_loader)

# Extract deep features and train XGBoost
def extract_features(model, dataloader):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for images, lbls in dataloader:
            outputs = model.feature_extractor(images)
            features.append(outputs.cpu().numpy())
            labels.append(lbls.cpu().numpy())
    return np.vstack(features), np.hstack(labels)

train_features, train_labels = extract_features(trained_model, train_loader)
test_features, test_labels = extract_features(trained_model, test_loader)

xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1)
xgb_model.fit(train_features, train_labels)
xgb_preds = xgb_model.predict(test_features)

final_accuracy = accuracy_score(test_labels, xgb_preds)
print(f"Final XGBoost Accuracy: {final_accuracy:.2%}")
